In [1]:
#!/usr/bin/python
# essential imports
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import timeit

import pandas as pd

from lda import evaluate_acc
from lda import lda
from lda import kfold_index
from lda import standardize
from lda import normalize
# chi2 user
from lda import chi2
from logreg import LogReg
# chi2 scikit
# from sklearn.feature_selection import chi2

# LDA test
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


# Automated test (normalized, first 11 features are kept)

In [5]:
#!/usr/bin/python
# essential imports
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import timeit

import pandas as pd

from lda import evaluate_acc
from lda import lda
from lda import kfold_index
from lda import standardize
from lda import normalize

# chi2 user
from lda import chi2
from logreg import LogReg

# chi2 scikit
# from sklearn.feature_selection import chi2

# LDA test
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


dlist = []
pvallist = [0.01, 0.05, 0.1, 0.25, 0.5, 1]
interactionT = [1, 2, 3, 4,5,6]

# Load datasets
# 1599 red win
wine_path = "./Data/winequality-red.csv"
data = np.genfromtxt(wine_path, delimiter=";", skip_header=1)

for interaction in interactionT:
    for pvali in pvallist:

        X_wine = data[:, 0:11]  # select columns 1 through 11
        y_wine = data[:, 11]

        # convert y values to 0, 1
        y_wine = np.where(y_wine < 5.5, 0, 1)

        # standardize
        # X_wine = standardize(X_wine)

        # adding interaction terms
        from itertools import combinations

        list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        comblist = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        comb = combinations(comblist, 2)

        i = 11
        if interaction in [2, 3, 4, 5,6]:
            for (a, b) in comb:
                # print((a, b), i)
                list.append((a, b))
                i += 1
                inter = X_wine[:, a] * X_wine[:, b]
                X_wine = np.column_stack((X_wine, inter))
            if interaction in [3, 4, 5,6]:
                comb3 = combinations(comblist, 3)
                for (a, b, c) in comb3:
                    # print((a, b, c), i)
                    list.append((a, b, c))
                    i += 1
                    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c]
                    X_wine = np.column_stack((X_wine, inter))
                if interaction in [4, 5,6]:
                    comb4 = combinations(comblist, 4)
                    for (a, b, c, d) in comb4:
                        # print((a, b, c, d), i)
                        list.append((a, b, c, d))
                        i += 1
                        inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]
                        X_wine = np.column_stack((X_wine, inter))
                    if interaction in [5,6]:
                        comb5 = combinations(comblist, 5)
                        for (a, b, c, d, e) in comb5:
                            # print((a, b, c, d, e), i)
                            list.append((a, b, c, d, e))
                            i += 1
                            inter = (
                                X_wine[:, a]
                                * X_wine[:, b]
                                * X_wine[:, c]
                                * X_wine[:, d]
                                * X_wine[:, e]
                            )
                            X_wine = np.column_stack((X_wine, inter))
                        if interaction == 6:
                            comb6 = combinations(comblist, 6)
                            for (a, b, c, d, e,f) in comb6:
                                # print((a, b, c, d, e,f), i)
                                list.append((a, b, c, d, e,f))
                                i += 1
                                inter = (
                                    X_wine[:, a]
                                    * X_wine[:, b]
                                    * X_wine[:, c]
                                    * X_wine[:, d]
                                    * X_wine[:, e]
                                    * X_wine[:, f]
                                )
                                X_wine = np.column_stack((X_wine, inter))
                            
        else:
            print("interaction is 1-way: ", interaction)

        # normalize
        X_wine = normalize(X_wine)

        # chi squared table todo
        chi, pval = chi2(X_wine, y_wine)
        list = np.asarray(list)
        chilist = np.column_stack((list, chi, pval))

        # threshold of chisquare interaction
        index_of_chi = np.where(pval > pvali)
        # keep 1-way interaction terms when testing 2 way or larger
        if interaction in [2, 3, 4, 5,6]:
            index_of_chi = np.delete(index_of_chi, np.where(index_of_chi[0] < 11), axis=1)
        # remove comment to delete insignificant interaction rows
        X_wine = np.delete(X_wine, index_of_chi, axis=1)
        chilist = np.delete(chilist, index_of_chi, axis=0)
        #print(chilist)
        print("number of features: ", len(chilist))
        print("p value: ", pvali)
        print("%d-way interaction" % interaction)
        y_wine = y_wine.reshape(-1, 1)



        # kfold test 10 times
        score = []
        for i in range(10):
            for train_index, test_index in kfold_index(5, X_wine):
                # print("TRAIN:", train_index, "TEST:", test_index)
                X_train, X_test = X_wine[train_index], X_wine[test_index]
                y_train, y_test = y_wine[train_index], y_wine[test_index]
                # project lda
                ld = LogReg(learning_rate=0.001)
                ld.fit(X_train, y_train)
                y_pred = ld.predict(X_test)
                score.append(evaluate_acc(y_test, y_pred))
        print("mean acc: ", np.mean(score))
        print("std acc: ", np.std(score))
        print()
        acc = np.mean(score)


        dlist.append(((pvali,interaction,chilist),acc))



#print(dlist)
from operator import itemgetter
print(max(dlist,key=itemgetter(1)))
((pvali,interaction,chilist),acc)=max(dlist,key=itemgetter(1))
print("best acc for below: ", acc)
print("number of features: ", len(chilist))
print("p value: ", pvali)
print("%d-way interaction" % interaction)
print("features chi/pval: ", chilist)

interaction is 1-way:  1
number of features:  3
p value:  0.01
1-way interaction
mean acc:  0.7378965517241381
std acc:  0.019820140659114742

interaction is 1-way:  1
number of features:  5
p value:  0.05
1-way interaction
mean acc:  0.7415848354231974
std acc:  0.01899379751806374

interaction is 1-way:  1
number of features:  5
p value:  0.1
1-way interaction
mean acc:  0.7422815438871473
std acc:  0.02568205174570555

interaction is 1-way:  1
number of features:  6
p value:  0.25
1-way interaction
mean acc:  0.7419110501567399
std acc:  0.02103671362408487

interaction is 1-way:  1
number of features:  9
p value:  0.5
1-way interaction
mean acc:  0.7427752742946709
std acc:  0.020312883719775764

interaction is 1-way:  1
number of features:  11
p value:  1
1-way interaction
mean acc:  0.7420916927899688
std acc:  0.017492340438041834

number of features:  24
p value:  0.01
2-way interaction
mean acc:  0.7427147335423198
std acc:  0.022670406240661117

number of features:  33
p valu

# Automated test (not normalized, first 11 features are kept)

In [25]:
#!/usr/bin/python
# essential imports
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import timeit

import pandas as pd

from lda import evaluate_acc
from lda import lda
from lda import kfold_index
from lda import standardize
from lda import normalize

# chi2 user
from lda import chi2
from logreg import LogReg

# chi2 scikit
# from sklearn.feature_selection import chi2

# LDA test
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


dlist = []
pvallist = [0.01, 0.05, 0.1, 0.25, 0.5, 1]
interactionT = [1, 2, 3, 4, 5,6]

# Load datasets
# 1599 red win
wine_path = "./Data/winequality-red.csv"
data = np.genfromtxt(wine_path, delimiter=";", skip_header=1)

for interaction in interactionT:
    for pvali in pvallist:

        X_wine = data[:, 0:11]  # select columns 1 through 11
        y_wine = data[:, 11]

        # convert y values to 0, 1
        y_wine = np.where(y_wine < 5.5, 0, 1)

        # standardize
        # X_wine = standardize(X_wine)

        # adding interaction terms
        from itertools import combinations

        list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        comblist = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        comb = combinations(comblist, 2)

        i = 11
        if interaction in [2, 3, 4, 5,6]:
            for (a, b) in comb:
                # print((a, b), i)
                list.append((a, b))
                i += 1
                inter = X_wine[:, a] * X_wine[:, b]
                X_wine = np.column_stack((X_wine, inter))
            if interaction in [3, 4, 5,6]:
                comb3 = combinations(comblist, 3)
                for (a, b, c) in comb3:
                    # print((a, b, c), i)
                    list.append((a, b, c))
                    i += 1
                    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c]
                    X_wine = np.column_stack((X_wine, inter))
                if interaction in [4, 5,6]:
                    comb4 = combinations(comblist, 4)
                    for (a, b, c, d) in comb4:
                        # print((a, b, c, d), i)
                        list.append((a, b, c, d))
                        i += 1
                        inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]
                        X_wine = np.column_stack((X_wine, inter))
                    if interaction in [5,6]:
                        comb5 = combinations(comblist, 5)
                        for (a, b, c, d, e) in comb5:
                            # print((a, b, c, d, e), i)
                            list.append((a, b, c, d, e))
                            i += 1
                            inter = (
                                X_wine[:, a]
                                * X_wine[:, b]
                                * X_wine[:, c]
                                * X_wine[:, d]
                                * X_wine[:, e]
                            )
                            X_wine = np.column_stack((X_wine, inter))
                        if interaction == 6:
                            comb6 = combinations(comblist, 6)
                            for (a, b, c, d, e,f) in comb6:
                                # print((a, b, c, d, e,f), i)
                                list.append((a, b, c, d, e,f))
                                i += 1
                                inter = (
                                    X_wine[:, a]
                                    * X_wine[:, b]
                                    * X_wine[:, c]
                                    * X_wine[:, d]
                                    * X_wine[:, e]
                                    * X_wine[:, f]
                                )
                                X_wine = np.column_stack((X_wine, inter))
                            
        else:
            print("interaction is 1-way: ", interaction)

        # normalize
        #X_wine = normalize(X_wine)

        # chi squared table todo
        chi, pval = chi2(X_wine, y_wine)
        list = np.asarray(list)
        chilist = np.column_stack((list, chi, pval))

        # threshold of chisquare interaction
        index_of_chi = np.where(pval > pvali)
        # keep 1-way interaction terms when testing 2 way or larger
        if interaction in [2, 3, 4, 5,6]:
            index_of_chi = np.delete(index_of_chi, np.where(index_of_chi[0] < 11), axis=1)
        # remove comment to delete insignificant interaction rows
        X_wine = np.delete(X_wine, index_of_chi, axis=1)
        chilist = np.delete(chilist, index_of_chi, axis=0)
        #print(chilist)
        print("number of features: ", len(chilist))
        print("p value: ", pvali)
        print("%d-way interaction" % interaction)
        y_wine = y_wine.reshape(-1, 1)



        # kfold test 1 times
        score = []
        for i in range(1):
            for train_index, test_index in kfold_index(5, X_wine):
                # print("TRAIN:", train_index, "TEST:", test_index)
                X_train, X_test = X_wine[train_index], X_wine[test_index]
                y_train, y_test = y_wine[train_index], y_wine[test_index]
                # project lda
                ld = LogReg(learning_rate=0.001)
                ld.fit(X_train, y_train)
                y_pred = ld.predict(X_test)
                score.append(evaluate_acc(y_test, y_pred))
        print("mean acc: ", np.mean(score))
        print("std acc: ", np.std(score))
        print()
        acc = np.mean(score)


        dlist.append(((pvali,interaction,chilist),acc))



#print(dlist)
from operator import itemgetter
print(max(dlist,key=itemgetter(1)))
((pvali,interaction,chilist),acc)=max(dlist,key=itemgetter(1))
print("best acc for below: ", acc)
print("number of features: ", len(chilist))
print("p value: ", pvali)
print("%d-way interaction" % interaction)
print("features chi/pval: ", chilist)

interaction is 1-way:  1
number of features:  4
p value:  0.01
1-way interaction
mean acc:  0.5553546238244514
std acc:  0.02012246974148718

interaction is 1-way:  1
number of features:  6
p value:  0.05
1-way interaction
mean acc:  0.5709737460815048
std acc:  0.02130120408507932

interaction is 1-way:  1
number of features:  7
p value:  0.1
1-way interaction
mean acc:  0.5684952978056426
std acc:  0.02071064057708918

interaction is 1-way:  1
number of features:  7
p value:  0.25
1-way interaction
mean acc:  0.570382053291536
std acc:  0.024957124345139563

interaction is 1-way:  1
number of features:  8
p value:  0.5
1-way interaction
mean acc:  0.5640987460815048
std acc:  0.024951934876776397

interaction is 1-way:  1
number of features:  11
p value:  1
1-way interaction
mean acc:  0.5728702978056426
std acc:  0.025767649821316664

number of features:  48
p value:  0.01
2-way interaction
mean acc:  0.563489420062696
std acc:  0.029772431630639605

number of features:  51
p value:

# individual test (normalized)
## no interactions (chi = 0.0000001) (74%)

In [20]:
# Load datasets
# 1599 red win
wine_path = "./Data/winequality-red.csv"
data = np.genfromtxt(wine_path, delimiter=";", skip_header=1)
# with open(wine_path) as f:
#     f.readline()
#     data = np.loadtxt(f, delimiter=";")
# f = open(wine_path)
# f.readline()  # skip the header
# data = np.loadtxt(f, delimiter=";")
# f.close()

X_wine = data[:, 0:11]  # select columns 1 through 11
y_wine = data[:, 11]

# convert y values to 0, 1
y_wine = np.where(y_wine < 5.5, 0, 1)
# print(y_wine[:100])
# wine = list(zip(X_wine, y_wine))
# print("Total number of wine", len(wine))
# print(X_wine[0])
# print(y_wine[0])
# print(wine[0])

# standardize
#X_wine = standardize(X_wine)

# adding interaction terms

from itertools import combinations

list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comblist = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comb = combinations(comblist, 2)

#i = 11
#for (a, b) in comb:
#    # print((a, b), i)
#    list.append((a, b))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b]
#    X_wine = np.column_stack((X_wine, inter))
#
#comb3 = combinations(comblist, 3)
#for (a, b, c) in comb3:
#    #print((a, b, c), i)
#    list.append((a, b, c))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c]
#    X_wine = np.column_stack((X_wine, inter))
#
#comb4 = combinations(comblist, 4)
#for (a, b, c, d) in comb4:
#    #print((a, b, c, d), i)
#    list.append((a, b, c, d))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]
#    X_wine = np.column_stack((X_wine, inter))
#
#comb5 = combinations(comblist, 5)
#for (a, b, c, d, e) in comb5:
#    #print((a, b, c, d, e), i)
#    list.append((a, b, c, d, e))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]  * X_wine[:, e]
#    X_wine = np.column_stack((X_wine, inter))
    

# normalize
X_wine = normalize(X_wine)

# chi squared table todo
chi, pval = chi2(X_wine, y_wine)
list = np.asarray(list)
chilist = np.column_stack((list, chi, pval))

# threshold of chisquare interaction
index_of_chi = np.where(pval > 0.05)
#keep no interaction terms
#index_of_chi = np.delete(index_of_chi, np.where(index_of_chi[0] < 11), axis=1)
# remove comment to delete insignificant interaction rows
X_wine = np.delete(X_wine, index_of_chi , axis=1)
chilist = np.delete(chilist, index_of_chi, axis=0)
print(chilist)
print("number of features: ",len(chilist))
# variance threshold on features
# print(X_wine.var(axis=0))
# for lda removing features decreased accuracy
# index_of_var = np.where(X_wine.var(axis=0) < 0.00000001)
# X_wine = np.delete(X_wine, index_of_var[0] , axis=1)

y_wine = y_wine.reshape(-1,1)
# recombininng and shuffling
#wine = np.column_stack((X_wine, y_wine))
#np.random.shuffle(wine)

# split into train test set
#n = X_wine.shape[0]
#f_size = X_wine.shape[1]
#X_wine = wine[:, 0:f_size]
#X_wine_test = wine[:, 0:f_size]
#y_wine = wine[:, f_size].reshape(-1,1)
#y_wine_test = wine[:, f_size].reshape(-1,1)

# standardize

#X_wine = normalize(X_wine)
#X_wine_test= normalize(X_wine_test)


# qualities = [float(item[-1]) for item in wines[1:]]
# sum(qualities) / len(qualities)


[[1.00000000e+00 8.89104331e+00 2.86572935e-03]
 [2.00000000e+00 5.66670287e+00 1.72899237e-02]
 [6.00000000e+00 8.12431060e+00 4.36755887e-03]
 [9.00000000e+00 3.98440125e+00 4.59234196e-02]
 [1.00000000e+01 2.60853253e+01 3.26657278e-07]]
number of features:  5


In [19]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  5
0.742024882445141


## 2 way interaction

In [2]:
# Load datasets
# 1599 red win
wine_path = "./Data/winequality-red.csv"
data = np.genfromtxt(wine_path, delimiter=";", skip_header=1)
# with open(wine_path) as f:
#     f.readline()
#     data = np.loadtxt(f, delimiter=";")
# f = open(wine_path)
# f.readline()  # skip the header
# data = np.loadtxt(f, delimiter=";")
# f.close()

X_wine = data[:, 0:11]  # select columns 1 through 11
y_wine = data[:, 11]

# convert y values to 0, 1
y_wine = np.where(y_wine < 5.5, 0, 1)
# print(y_wine[:100])
# wine = list(zip(X_wine, y_wine))
# print("Total number of wine", len(wine))
# print(X_wine[0])
# print(y_wine[0])
# print(wine[0])

# standardize
#X_wine = standardize(X_wine)

# adding interaction terms

from itertools import combinations

list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comblist = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comb = combinations(comblist, 2)

i = 11
for (a, b) in comb:
    # print((a, b), i)
    list.append((a, b))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b]
    X_wine = np.column_stack((X_wine, inter))
#
#comb3 = combinations(comblist, 3)
#for (a, b, c) in comb3:
#    #print((a, b, c), i)
#    list.append((a, b, c))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c]
#    X_wine = np.column_stack((X_wine, inter))
#
#comb4 = combinations(comblist, 4)
#for (a, b, c, d) in comb4:
#    #print((a, b, c, d), i)
#    list.append((a, b, c, d))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]
#    X_wine = np.column_stack((X_wine, inter))
#
#comb5 = combinations(comblist, 5)
#for (a, b, c, d, e) in comb5:
#    #print((a, b, c, d, e), i)
#    list.append((a, b, c, d, e))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]  * X_wine[:, e]
#    X_wine = np.column_stack((X_wine, inter))
    

# normalize
X_wine = normalize(X_wine)

# chi squared table todo
chi, pval = chi2(X_wine, y_wine)
list = np.asarray(list)
chilist = np.column_stack((list, chi, pval))

# threshold of chisquare interaction
index_of_chi = np.where(pval < 0.05)
#keep no interaction terms
index_of_chi = np.delete(index_of_chi, np.where(index_of_chi[0] < 11), axis=1)
# remove comment to delete insignificant interaction rows
#X_wine = np.delete(X_wine, index_of_chi , axis=1)
chilist = np.delete(chilist, index_of_chi, axis=0)
print(chilist)
print("number of features: ",len(chilist))

# variance threshold on features
# print(X_wine.var(axis=0))
# for lda removing features decreased accuracy
# index_of_var = np.where(X_wine.var(axis=0) < 0.00000001)
# X_wine = np.delete(X_wine, index_of_var[0] , axis=1)


# recombininng and shuffling
wine = np.column_stack((X_wine, y_wine))
#np.random.shuffle(wine)

# split into train test set
n = X_wine.shape[0]
f_size = X_wine.shape[1]
X_wine = wine[(n // 10) :, 0:f_size]
X_wine_test = wine[: (n // 10), 0:f_size]
y_wine = wine[(n // 10) :, f_size].reshape(-1,1)
y_wine_test = wine[: (n // 10), f_size].reshape(-1,1)

# standardize

#X_wine = normalize(X_wine)
#X_wine_test= normalize(X_wine_test)


# qualities = [float(item[-1]) for item in wines[1:]]
# sum(qualities) / len(qualities)


[[0 1.0421852090088097 0.3073137941896506]
 [1 8.891043306316842 0.0028657293536039487]
 [2 5.666702872536871 0.017289923715079227]
 [3 0.0006196962034192414 0.980139753892595]
 [4 0.93881024263718 0.33258400473573235]
 [5 0.6314063119772613 0.42683993824974265]
 [6 8.124310600120374 0.004367558871853705]
 [7 1.5846430385807455 0.20809316689519622]
 [8 0.0005594550998740942 0.9811295530403128]
 [9 3.9844012485431906 0.04592341963360741]
 [10 26.08532534641509 3.266572782179879e-07]
 [(0, 1) 7.326107125644484 0.006796008852444809]
 [(0, 2) 6.326521347333127 0.011894540260780607]
 [(0, 6) 7.65742921374715 0.005653901097783527]
 [(0, 9) 5.930387759132255 0.01488196682923952]
 [(0, 10) 6.439397254546957 0.011161651229169178]
 [(1, 3) 3.5662748142960847 0.05896444875823862]
 [(1, 4) 6.56301660386709 0.010411971648554024]
 [(1, 5) 5.411228701708492 0.02000762852905959]
 [(1, 6) 15.512383994186933 8.196649778217143e-05]
 [(1, 7) 8.949767167109705 0.0027750495281751927]
 [(1, 8) 8.388006917260

In [120]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  39
0.745138888888889


In [55]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  66
0.7454166666666666


## 3 way interaction (highest 74.7%)

In [117]:
# Load datasets
# 1599 red win
wine_path = "./Data/winequality-red.csv"
data = np.genfromtxt(wine_path, delimiter=";", skip_header=1)
# with open(wine_path) as f:
#     f.readline()
#     data = np.loadtxt(f, delimiter=";")
# f = open(wine_path)
# f.readline()  # skip the header
# data = np.loadtxt(f, delimiter=";")
# f.close()

X_wine = data[:, 0:11]  # select columns 1 through 11
y_wine = data[:, 11]

# convert y values to 0, 1
y_wine = np.where(y_wine < 5.5, 0, 1)
# print(y_wine[:100])
# wine = list(zip(X_wine, y_wine))
# print("Total number of wine", len(wine))
# print(X_wine[0])
# print(y_wine[0])
# print(wine[0])

# standardize
#X_wine = standardize(X_wine)

# adding interaction terms

from itertools import combinations

list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comblist = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comb = combinations(comblist, 2)

i = 11
for (a, b) in comb:
    # print((a, b), i)
    list.append((a, b))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b]
    X_wine = np.column_stack((X_wine, inter))

comb3 = combinations(comblist, 3)
for (a, b, c) in comb3:
    #print((a, b, c), i)
    list.append((a, b, c))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c]
    X_wine = np.column_stack((X_wine, inter))

#comb4 = combinations(comblist, 4)
#for (a, b, c, d) in comb4:
#    #print((a, b, c, d), i)
#    list.append((a, b, c, d))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]
#    X_wine = np.column_stack((X_wine, inter))
#
#comb5 = combinations(comblist, 5)
#for (a, b, c, d, e) in comb5:
#    #print((a, b, c, d, e), i)
#    list.append((a, b, c, d, e))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]  * X_wine[:, e]
#    X_wine = np.column_stack((X_wine, inter))
    

# normalize
X_wine = normalize(X_wine)

# chi squared table todo
chi, pval = chi2(X_wine, y_wine)
list = np.asarray(list)
chilist = np.column_stack((list, chi, pval))

# threshold of chisquare interaction
index_of_chi = np.where(chi < 0.0001)
#keep no interaction terms
index_of_chi = np.delete(index_of_chi, np.where(index_of_chi[0] < 11), axis=1)
# remove comment to delete insignificant interaction rows
#X_wine = np.delete(X_wine, index_of_chi , axis=1)
chilist = np.delete(chilist, index_of_chi, axis=0)
print(chilist)
print(len(chilist))

# variance threshold on features
# print(X_wine.var(axis=0))
# for lda removing features decreased accuracy
# index_of_var = np.where(X_wine.var(axis=0) < 0.00000001)
# X_wine = np.delete(X_wine, index_of_var[0] , axis=1)


# recombininng and shuffling
wine = np.column_stack((X_wine, y_wine))
#np.random.shuffle(wine)

# split into train test set
n = X_wine.shape[0]
f_size = X_wine.shape[1]
X_wine = wine[(n // 10) :, 0:f_size]
X_wine_test = wine[: (n // 10), 0:f_size]
y_wine = wine[(n // 10) :, f_size].reshape(-1,1)
y_wine_test = wine[: (n // 10), f_size].reshape(-1,1)

# standardize

#X_wine = normalize(X_wine)
#X_wine_test= normalize(X_wine_test)


# qualities = [float(item[-1]) for item in wines[1:]]
# sum(qualities) / len(qualities)


[[0 1.0421852090088097 0.3073137941896506]
 [1 8.891043306316842 0.0028657293536039487]
 [2 5.666702872536871 0.017289923715079227]
 [3 0.0006196962034192414 0.980139753892595]
 [4 0.93881024263718 0.33258400473573235]
 [5 0.6314063119772613 0.42683993824974265]
 [6 8.124310600120374 0.004367558871853705]
 [7 1.5846430385807455 0.20809316689519622]
 [8 0.0005594550998740942 0.9811295530403128]
 [9 3.9844012485431906 0.04592341963360741]
 [10 26.08532534641509 3.266572782179879e-07]
 [(0, 1) 7.326107125644484 0.006796008852444809]
 [(0, 2) 6.326521347333127 0.011894540260780607]
 [(0, 3) 0.11449164602399525 0.7350872857384305]
 [(0, 4) 0.3471371653723725 0.5557382022901459]
 [(0, 5) 0.37313859717896325 0.5412984146053782]
 [(0, 6) 7.65742921374715 0.005653901097783527]
 [(0, 7) 1.0095282109476136 0.3150158915915632]
 [(0, 8) 1.4343251181540113 0.23105987186601448]
 [(0, 9) 5.930387759132255 0.01488196682923952]
 [(0, 10) 6.439397254546957 0.011161651229169178]
 [(1, 2) 0.001675624298951

In [118]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  231
0.7468750000000002


In [113]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  165
0.7460416666666667


In [116]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  116
0.747638888888889


## 4 way interaction

In [107]:
# Load datasets
# 1599 red win
wine_path = "./Data/winequality-red.csv"
data = np.genfromtxt(wine_path, delimiter=";", skip_header=1)
# with open(wine_path) as f:
#     f.readline()
#     data = np.loadtxt(f, delimiter=";")
# f = open(wine_path)
# f.readline()  # skip the header
# data = np.loadtxt(f, delimiter=";")
# f.close()

X_wine = data[:, 0:11]  # select columns 1 through 11
y_wine = data[:, 11]

# convert y values to 0, 1
y_wine = np.where(y_wine < 5.5, 0, 1)
# print(y_wine[:100])
# wine = list(zip(X_wine, y_wine))
# print("Total number of wine", len(wine))
# print(X_wine[0])
# print(y_wine[0])
# print(wine[0])

# standardize
#X_wine = standardize(X_wine)

# adding interaction terms

from itertools import combinations

list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comblist = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comb = combinations(comblist, 2)

i = 11
for (a, b) in comb:
    # print((a, b), i)
    list.append((a, b))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b]
    X_wine = np.column_stack((X_wine, inter))

comb3 = combinations(comblist, 3)
for (a, b, c) in comb3:
    #print((a, b, c), i)
    list.append((a, b, c))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c]
    X_wine = np.column_stack((X_wine, inter))

comb4 = combinations(comblist, 4)
for (a, b, c, d) in comb4:
    #print((a, b, c, d), i)
    list.append((a, b, c, d))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]
    X_wine = np.column_stack((X_wine, inter))
#
#comb5 = combinations(comblist, 5)
#for (a, b, c, d, e) in comb5:
#    #print((a, b, c, d, e), i)
#    list.append((a, b, c, d, e))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]  * X_wine[:, e]
#    X_wine = np.column_stack((X_wine, inter))
    

# normalize
X_wine = normalize(X_wine)

# chi squared table todo
chi, pval = chi2(X_wine, y_wine)
list = np.asarray(list)
chilist = np.column_stack((list, chi, pval))

# threshold of chisquare interaction
index_of_chi = np.where(chi < 3)
#keep no interaction terms
index_of_chi = np.delete(index_of_chi, np.where(index_of_chi[0] < 11), axis=1)
# remove comment to delete insignificant interaction rows
X_wine = np.delete(X_wine, index_of_chi , axis=1)
chilist = np.delete(chilist, index_of_chi, axis=0)
print(chilist)
print("number of features: ",len(chilist))

# variance threshold on features
# print(X_wine.var(axis=0))
# for lda removing features decreased accuracy
# index_of_var = np.where(X_wine.var(axis=0) < 0.00000001)
# X_wine = np.delete(X_wine, index_of_var[0] , axis=1)


# recombininng and shuffling
wine = np.column_stack((X_wine, y_wine))
#np.random.shuffle(wine)

# split into train test set
n = X_wine.shape[0]
f_size = X_wine.shape[1]
X_wine = wine[(n // 10) :, 0:f_size]
X_wine_test = wine[: (n // 10), 0:f_size]
y_wine = wine[(n // 10) :, f_size].reshape(-1,1)
y_wine_test = wine[: (n // 10), f_size].reshape(-1,1)

# standardize

#X_wine = normalize(X_wine)
#X_wine_test= normalize(X_wine_test)


# qualities = [float(item[-1]) for item in wines[1:]]
# sum(qualities) / len(qualities)


[[0 1.042185209008817 0.3073137941896489]
 [1 8.891043306316842 0.0028657293536039487]
 [2 5.666702872536845 0.01728992371507948]
 [3 0.0006196962034192906 0.9801397538925941]
 [4 0.93881024263718 0.33258400473573235]
 [5 0.6314063119772713 0.4268399382497391]
 [6 8.124310600120378 0.0043675588718536905]
 [7 1.5846430385807617 0.20809316689519386]
 [8 0.0005594550998740942 0.9811295530403128]
 [9 3.9844012485431595 0.045923419633608194]
 [10 26.08532534641502 3.2665727821800035e-07]
 [(0, 1) 7.326107125644496 0.0067960088524447555]
 [(0, 2) 6.326521347333114 0.01189454026078067]
 [(0, 6) 7.657429213747134 0.005653901097783591]
 [(0, 9) 5.930387759132255 0.01488196682923952]
 [(0, 10) 6.43939725454695 0.011161651229169232]
 [(1, 3) 3.5662748142960856 0.0589644487582384]
 [(1, 4) 6.563016603867104 0.010411971648553951]
 [(1, 5) 5.41122870170849 0.02000762852905962]
 [(1, 6) 15.512383994186958 8.196649778217025e-05]
 [(1, 7) 8.949767167109703 0.0027750495281751927]
 [(1, 8) 8.388006917260

In [66]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  561
0.7388194444444445


In [62]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  331
0.7499305555555555


In [64]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  252
0.750486111111111


In [108]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  194
0.750138888888889


## 5 way interaction (notice increase in accuracy to 75.1%)
## best accuracy around 300 features (chi < 2)

In [104]:
# Load datasets
# 1599 red win
wine_path = "./Data/winequality-red.csv"
data = np.genfromtxt(wine_path, delimiter=";", skip_header=1)
# with open(wine_path) as f:
#     f.readline()
#     data = np.loadtxt(f, delimiter=";")
# f = open(wine_path)
# f.readline()  # skip the header
# data = np.loadtxt(f, delimiter=";")
# f.close()

X_wine = data[:, 0:11]  # select columns 1 through 11
y_wine = data[:, 11]

# convert y values to 0, 1
y_wine = np.where(y_wine < 5.5, 0, 1)
# print(y_wine[:100])
# wine = list(zip(X_wine, y_wine))
# print("Total number of wine", len(wine))
# print(X_wine[0])
# print(y_wine[0])
# print(wine[0])

# standardize
#X_wine = standardize(X_wine)

# adding interaction terms

from itertools import combinations

list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comblist = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comb = combinations(comblist, 2)

i = 11
for (a, b) in comb:
    # print((a, b), i)
    list.append((a, b))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b]
    X_wine = np.column_stack((X_wine, inter))

comb3 = combinations(comblist, 3)
for (a, b, c) in comb3:
    #print((a, b, c), i)
    list.append((a, b, c))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c]
    X_wine = np.column_stack((X_wine, inter))

comb4 = combinations(comblist, 4)
for (a, b, c, d) in comb4:
    #print((a, b, c, d), i)
    list.append((a, b, c, d))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]
    X_wine = np.column_stack((X_wine, inter))

comb5 = combinations(comblist, 5)
for (a, b, c, d, e) in comb5:
    #print((a, b, c, d, e), i)
    list.append((a, b, c, d, e))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]  * X_wine[:, e]
    X_wine = np.column_stack((X_wine, inter))
    

# normalize
X_wine = normalize(X_wine)

# chi squared table todo
chi, pval = chi2(X_wine, y_wine)
list = np.asarray(list)
chilist = np.column_stack((list, chi, pval))

# threshold of chisquare interaction
index_of_chi = np.where(chi < 2)
#keep no interaction terms
index_of_chi = np.delete(index_of_chi, np.where(index_of_chi[0] < 11), axis=1)
# remove comment to delete insignificant interaction rows
X_wine = np.delete(X_wine, index_of_chi , axis=1)
chilist = np.delete(chilist, index_of_chi, axis=0)
print(chilist)
print(len(chilist))

# variance threshold on features
# print(X_wine.var(axis=0))
# for lda removing features decreased accuracy
# index_of_var = np.where(X_wine.var(axis=0) < 0.00000001)
# X_wine = np.delete(X_wine, index_of_var[0] , axis=1)


# recombininng and shuffling
wine = np.column_stack((X_wine, y_wine))
#np.random.shuffle(wine)

# split into train test set
n = X_wine.shape[0]
f_size = X_wine.shape[1]
X_wine = wine[(n // 10) :, 0:f_size]
X_wine_test = wine[: (n // 10), 0:f_size]
y_wine = wine[(n // 10) :, f_size].reshape(-1,1)
y_wine_test = wine[: (n // 10), f_size].reshape(-1,1)

# standardize

#X_wine = normalize(X_wine)
#X_wine_test= normalize(X_wine_test)


# qualities = [float(item[-1]) for item in wines[1:]]
# sum(qualities) / len(qualities)


[[0 1.042185209008817 0.3073137941896489]
 [1 8.891043306316842 0.0028657293536039487]
 [2 5.666702872536845 0.01728992371507948]
 ...
 [(5, 6, 7, 8, 10) 2.9813658363089584 0.08422816680819549]
 [(5, 6, 7, 9, 10) 2.0649364972954003 0.15072120702755837]
 [(5, 6, 8, 9, 10) 2.0299253126784547 0.15422818633148064]]
413


In [74]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  874
0.7279861111111111


In [68]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  580
0.7504166666666666


In [105]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  413
0.7509722222222222


In [103]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  315
0.7486111111111111


In [70]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  253
0.7473611111111111


In [72]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

#print(score)
# print(scorec)
print(np.mean(score))
#print(np.mean(scorec))
# test runtime
# timeit.timeit(ld.fit(X_train, y_train), number=10)??

number of features:  213
0.7470833333333334


# removing features 0,3,4,5,7,8

In [88]:
# Load datasets
# 1599 red win
wine_path = "./Data/winequality-red.csv"
data = np.genfromtxt(wine_path, delimiter=";", skip_header=1)

X_wine = data[:, [1,2,6,9,10]]  # select columns 1 through 11
y_wine = data[:, 11]

# convert y values to 0, 1
y_wine = np.where(y_wine < 5.5, 0, 1)

# standardize
#X_wine = standardize(X_wine)

# adding interaction terms

from itertools import combinations
list = [0,1,2,3,4]
comblist = [0,1,2,3,4]
comb = combinations(comblist, 2)

#i = 7
#for (a, b) in comb:
#    # print((a, b), i)
#    list.append((a, b))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b]
#    X_wine = np.column_stack((X_wine, inter))

#comb3 = combinations(comblist, 3)
#for (a, b, c) in comb3:
#    #print((a, b, c), i)
#    list.append((a, b, c))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c]
#    X_wine = np.column_stack((X_wine, inter))

#comb4 = combinations(comblist, 4)
#for (a, b, c, d) in comb4:
#    #print((a, b, c, d), i)
#    list.append((a, b, c, d))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]
#    X_wine = np.column_stack((X_wine, inter))

#comb5 = combinations(comblist, 5)
#for (a, b, c, d, e) in comb5:
#    #print((a, b, c, d, e), i)
#    list.append((a, b, c, d, e))
#    i += 1
#    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]  * X_wine[:, e]
#    X_wine = np.column_stack((X_wine, inter))
    

# normalize
X_wine = normalize(X_wine)

# chi squared table todo
chi, pval = chi2(X_wine, y_wine)
list = np.asarray(list)
chilist = np.column_stack((list, chi, pval))

# threshold of chisquare interaction
index_of_chi = np.where(chi < 1)
#keep no interaction terms
index_of_chi = np.delete(index_of_chi, np.where(index_of_chi[0] < 5), axis=1)
# remove comment to delete insignificant interaction rows
X_wine = np.delete(X_wine, index_of_chi , axis=1)
chilist = np.delete(chilist, index_of_chi, axis=0)
print(chilist)
print("number of features: ",len(chilist))
print(X_wine.shape[1])


# recombininng and shuffling
wine = np.column_stack((X_wine, y_wine))
#np.random.shuffle(wine)

# split into train test set
n = X_wine.shape[0]
f_size = X_wine.shape[1]
X_wine = wine[:, 0:f_size]
#X_wine_test = wine[:, 0:f_size]
y_wine = wine[:, f_size].reshape(-1,1)
#y_wine_test = wine[:, f_size].reshape(-1,1)

# standardize

#X_wine = normalize(X_wine)
#X_wine_test= normalize(X_wine_test)

[[0.00000000e+00 8.89104331e+00 2.86572935e-03]
 [1.00000000e+00 5.66670287e+00 1.72899237e-02]
 [2.00000000e+00 8.12431060e+00 4.36755887e-03]
 [3.00000000e+00 3.98440125e+00 4.59234196e-02]
 [4.00000000e+00 2.60853253e+01 3.26657278e-07]]
number of features:  5
5


In [89]:
print("number of features with chi1: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

print(np.mean(score))
#print(np.mean(scorec))

number of features with chi1:  5
0.7410846394984325


In [87]:
print("number of features with chi1: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

print(np.mean(score))
#print(np.mean(scorec))

number of features with chi1:  19
0.740651645768025


In [85]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

print(np.mean(score))
#print(np.mean(scorec))

number of features:  15
0.7410325235109718


In [83]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

print(np.mean(score))
#print(np.mean(scorec))

number of features:  25
0.740587382445141


In [81]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

print(np.mean(score))
#print(np.mean(scorec))

number of features:  31
0.7383336598746082


# 6 way interaction similar to 5-way best 75.2% at chi = 2

In [99]:
# Load datasets
# 1599 red win
wine_path = "./Data/winequality-red.csv"
data = np.genfromtxt(wine_path, delimiter=";", skip_header=1)
# with open(wine_path) as f:
#     f.readline()
#     data = np.loadtxt(f, delimiter=";")
# f = open(wine_path)
# f.readline()  # skip the header
# data = np.loadtxt(f, delimiter=";")
# f.close()

X_wine = data[:, 0:11]  # select columns 1 through 11
y_wine = data[:, 11]

# convert y values to 0, 1
y_wine = np.where(y_wine < 5.5, 0, 1)
# print(y_wine[:100])
# wine = list(zip(X_wine, y_wine))
# print("Total number of wine", len(wine))
# print(X_wine[0])
# print(y_wine[0])
# print(wine[0])

# standardize
#X_wine = standardize(X_wine)

# adding interaction terms

from itertools import combinations

list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comblist = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
comb = combinations(comblist, 2)

i = 11
for (a, b) in comb:
    # print((a, b), i)
    list.append((a, b))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b]
    X_wine = np.column_stack((X_wine, inter))

comb3 = combinations(comblist, 3)
for (a, b, c) in comb3:
    #print((a, b, c), i)
    list.append((a, b, c))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c]
    X_wine = np.column_stack((X_wine, inter))

comb4 = combinations(comblist, 4)
for (a, b, c, d) in comb4:
    #print((a, b, c, d), i)
    list.append((a, b, c, d))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]
    X_wine = np.column_stack((X_wine, inter))

comb5 = combinations(comblist, 5)
for (a, b, c, d, e) in comb5:
    #print((a, b, c, d, e), i)
    list.append((a, b, c, d, e))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]  * X_wine[:, e]
    X_wine = np.column_stack((X_wine, inter))

comb6 = combinations(comblist, 6)
for (a, b, c, d, e, f) in comb6:
    #print((a, b, c, d, e, f), i)
    list.append((a, b, c, d, e, f))
    i += 1
    inter = X_wine[:, a] * X_wine[:, b] * X_wine[:, c] * X_wine[:, d]  * X_wine[:, e] * X_wine[:, f]
    X_wine = np.column_stack((X_wine, inter))
    

# normalize
X_wine = normalize(X_wine)

# chi squared table todo
chi, pval = chi2(X_wine, y_wine)
list = np.asarray(list)
chilist = np.column_stack((list, chi, pval))

# threshold of chisquare interaction
index_of_chi = np.where(chi < 4)
#keep no interaction terms
index_of_chi = np.delete(index_of_chi, np.where(index_of_chi[0] < 11), axis=1)
# remove comment to delete insignificant interaction rows
X_wine = np.delete(X_wine, index_of_chi , axis=1)
chilist = np.delete(chilist, index_of_chi, axis=0)
print(chilist)
print(len(chilist))

# variance threshold on features
# print(X_wine.var(axis=0))
# for lda removing features decreased accuracy
# index_of_var = np.where(X_wine.var(axis=0) < 0.00000001)
# X_wine = np.delete(X_wine, index_of_var[0] , axis=1)


# recombininng and shuffling
wine = np.column_stack((X_wine, y_wine))
#np.random.shuffle(wine)

# split into train test set
n = X_wine.shape[0]
f_size = X_wine.shape[1]
X_wine = wine[(n // 10) :, 0:f_size]
X_wine_test = wine[: (n // 10), 0:f_size]
y_wine = wine[(n // 10) :, f_size].reshape(-1,1)
y_wine_test = wine[: (n // 10), f_size].reshape(-1,1)

# standardize

#X_wine = normalize(X_wine)
#X_wine_test= normalize(X_wine_test)


# qualities = [float(item[-1]) for item in wines[1:]]
# sum(qualities) / len(qualities)


[[0 1.042185209008817 0.3073137941896489]
 [1 8.891043306316842 0.0028657293536039487]
 [2 5.666702872536845 0.01728992371507948]
 [3 0.0006196962034192906 0.9801397538925941]
 [4 0.93881024263718 0.33258400473573235]
 [5 0.6314063119772713 0.4268399382497391]
 [6 8.124310600120378 0.0043675588718536905]
 [7 1.5846430385807617 0.20809316689519386]
 [8 0.0005594550998740942 0.9811295530403128]
 [9 3.9844012485431595 0.045923419633608194]
 [10 26.08532534641502 3.2665727821800035e-07]
 [(0, 1) 7.326107125644496 0.0067960088524447555]
 [(0, 2) 6.326521347333114 0.01189454026078067]
 [(0, 6) 7.657429213747134 0.005653901097783591]
 [(0, 9) 5.930387759132255 0.01488196682923952]
 [(0, 10) 6.43939725454695 0.011161651229169232]
 [(1, 4) 6.563016603867104 0.010411971648553951]
 [(1, 5) 5.41122870170849 0.02000762852905962]
 [(1, 6) 15.512383994186958 8.196649778217025e-05]
 [(1, 7) 8.949767167109703 0.0027750495281751927]
 [(1, 8) 8.388006917260578 0.003777048435039452]
 [(2, 7) 5.62238689972

In [91]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

print(np.mean(score))
#print(np.mean(scorec))

number of features:  1483
0.6978472222222223


In [93]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

print(np.mean(score))
#print(np.mean(scorec))

number of features:  819
0.7415277777777779


In [95]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

print(np.mean(score))
#print(np.mean(scorec))

number of features:  559
0.751736111111111


In [98]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

print(np.mean(score))
#print(np.mean(scorec))

number of features:  427
0.7516666666666666


In [100]:
print("number of features: ",len(chilist))
# kfold test
score = []
scorec = []
for i in range(10):
    for train_index, test_index in kfold_index(5, X_wine):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_wine[train_index], X_wine[test_index]
        y_train, y_test = y_wine[train_index], y_wine[test_index]
        # project lda
        ld = LogReg(learning_rate=0.001)
        #ld = LogisticRegression(penalty = 'none', solver = 'sag', random_state=0, C = 1000)
        
        ld.fit(X_train, y_train)

        # scikit learn lda
        #clf = LinearDiscriminantAnalysis()
        #clf.fit(X_train, y_train)
        #y_predT = clf.predict(X_test)
        #scorec.append(evaluate_acc(y_test, y_predT))

        y_pred = ld.predict(X_test)
        score.append(evaluate_acc(y_test, y_pred))

print(np.mean(score))
#print(np.mean(scorec))

number of features:  330
0.7491666666666668
